In [34]:
import random
import subprocess

def process_video(input_file, output_file):
    """
    使用 FFmpeg 对视频进行进一步处理以规避重复内容审核：
    1. 移除元数据。
    2. 调整分辨率到 1280x720，随机裁剪或加边框。
    3. 调整帧率到 30fps。
    4. 添加动态亮度、对比度调整和随机噪声。
    5. 添加透明水印。
    6. 处理音频：调整音频速度和混响。
    """
    # 随机位置水印
    watermark_x = random.randint(0, 1280 - 200)  # 随机 X 位置
    watermark_y = random.randint(0, 720 - 50)   # 随机 Y 位置
    watermark_opacity = random.uniform(0.1, 0.3)  # 随机透明度

    # 随机色彩调整
    brightness = random.uniform(-0.05, 0.05)  # 随机亮度
    contrast = random.uniform(0.95, 1.05)     # 随机对比度
    saturation = random.uniform(0.95, 1.05)   # 随机饱和度

    # 创建 FFmpeg 命令
    command = [
        "ffmpeg", "-i", input_file,
        # 移除元数据
        "-map_metadata", "-1",
        # 视频滤镜：调整分辨率、亮度对比、随机裁剪、加边框、添加噪声和水印
        "-vf", f"scale=1280:720,eq=brightness={brightness}:contrast={contrast}:saturation={saturation},"
               f"noise=alls=10:allf=t+u,"
               f"drawtext=text='0.007':fontcolor=white@{watermark_opacity}:fontsize=24:x={watermark_x}:y={watermark_y}",
        # 设置帧率
        "-r", "30",
        # 视频编码器和参数
        "-c:v", "libx264", "-crf", "18", "-b:v", "3M", "-preset", "slow",
        # 音频滤镜：调整音频速度和音量
        "-filter:a", "atempo=1.05,volume=1.1",
        # 音频编码器
        "-c:a", "aac",
        # 视频剪辑（轻微调整）
        "-ss", "00:00:01",  # 从 5 秒开始
        "-t", "00:03:00",   # 保留 3 分钟
        # 轻微加速
        "-filter:v", "setpts=1.05*PTS",  # 视频加速 5%
        "-filter:a", "atempo=1.05",      # 音频同步
        # 输出文件
        output_file
    ]

    # 执行 FFmpeg 命令
    try:
        subprocess.run(command, check=True)
        print(f"视频处理完成，保存为: {output_file}")
    except subprocess.CalledProcessError as e:
        print(f"处理视频时出错: {e}")

# 示例用法
process_video("input_video.mp4", "output_video.mp4")


处理视频时出错: Command '['ffmpeg', '-i', 'input_video.mp4', '-map_metadata', '-1', '-vf', "scale=1280:720,eq=brightness=-0.018993291047319106:contrast=0.9614731647542711:saturation=0.9737997919518507,noise=alls=10:allf=t+u,drawtext=text='0.007':fontcolor=white@0.17613571906437722:fontsize=24:x=884:y=350", '-r', '30', '-c:v', 'libx264', '-crf', '18', '-b:v', '3M', '-preset', 'slow', '-filter:a', 'atempo=1.05,volume=1.1', '-c:a', 'aac', '-ss', '00:00:01', '-t', '00:03:00', '-filter:v', 'setpts=1.05*PTS', '-filter:a', 'atempo=1.05', 'output_video.mp4']' returned non-zero exit status 254.


ffmpeg version 7.1 Copyright (c) 2000-2024 the FFmpeg developers
  built with Apple clang version 16.0.0 (clang-1600.0.26.4)
  configuration: --prefix=/opt/homebrew/Cellar/ffmpeg/7.1_3 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags='-Wl,-ld_classic' --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libaribb24 --enable-libbluray --enable-libdav1d --enable-libharfbuzz --enable-libjxl --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libssh --enable-libsvtav1 --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libspeex --e

In [35]:
import glob
webm_files = glob.glob("./downloads/*.webm")
if webm_files:
    input_file = webm_files[0]
else:
    input_file = None
print(input_file)


output_file = input_file.replace(".webm", ".mp4")
process_video(input_file, output_file)



./downloads/NBA’s Top 5 Plays of the Night ｜ November 23, 2024.webm
处理视频时出错: Command '['ffmpeg', '-i', './downloads/NBA’s Top 5 Plays of the Night ｜ November 23, 2024.webm', '-map_metadata', '-1', '-vf', "scale=1280:720,eq=brightness=-0.012682658359582981:contrast=0.9809015709218387:saturation=0.955701444098857,noise=alls=10:allf=t+u,drawtext=text='0.007':fontcolor=white@0.1296003598938273:fontsize=24:x=103:y=443", '-r', '30', '-c:v', 'libx264', '-crf', '18', '-b:v', '3M', '-preset', 'slow', '-filter:a', 'atempo=1.05,volume=1.1', '-c:a', 'aac', '-ss', '00:00:01', '-t', '00:03:00', '-filter:v', 'setpts=1.05*PTS', '-filter:a', 'atempo=1.05', './processed/output.mp4']' returned non-zero exit status 254.


ffmpeg version 7.1 Copyright (c) 2000-2024 the FFmpeg developers
  built with Apple clang version 16.0.0 (clang-1600.0.26.4)
  configuration: --prefix=/opt/homebrew/Cellar/ffmpeg/7.1_3 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags='-Wl,-ld_classic' --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libaribb24 --enable-libbluray --enable-libdav1d --enable-libharfbuzz --enable-libjxl --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libssh --enable-libsvtav1 --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libspeex --e